In [ ]:
from helper.read_mongo_collection import HelperToReadMongo
from helper.customer_info import CustomerInfo

import pandas as pd

import os
import numpy as np
np.set_printoptions(threshold=np.inf)
import matplotlib.pyplot as plt
import statistics

import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

from matplotlib import pyplot as plt
from plotly.subplots import make_subplots

sns.set_theme(style="darkgrid")

In [ ]:
helper_to_read_mongo = HelperToReadMongo()
customer_info = CustomerInfo()

storage_path = "/mnt/d/PowerTAC/Demand_Response/Analysis/Varying_Peak_Rate_Within_Game/"
# storage_path = "/mnt/d/PowerTAC/Demand_Response/Analysis/Same_Peak_Rate/"
os.makedirs(storage_path, exist_ok=True)

In [ ]:
def get_dataframe(collection, customer):
    
    test_database = collection

    dataframe_calendar = helper_to_read_mongo.query_to_mongo(test_database, 'Calendar_Info', server_ip='<ip>', ssh_username='<username>', ssh_password='<password>', remote=True)
    dataframe_customer = helper_to_read_mongo.query_to_mongo(test_database, customer, server_ip='<ip>', ssh_username='<username>', ssh_password='<password>', remote=True)
    dataframe_distribution = helper_to_read_mongo.query_to_mongo(test_database, 'DistributionTransaction_and_Report_Info', server_ip='<ip>', ssh_username='<username>', ssh_password='<password>', remote=True)
    dataframe_capacity = helper_to_read_mongo.query_to_mongo(test_database, 'CapacityTransaction_Info', server_ip='<ip>', ssh_username='<username>', ssh_password='<password>', remote=True)
    
#     dataframe_calendar = helper_to_read_mongo.query_to_mongo(test_database, 'Calendar_Info')
#     dataframe_customer = helper_to_read_mongo.query_to_mongo(test_database, customer)

    dataframe_customer.columns = dataframe_customer.columns.str.replace(' ','_')

    df = pd.merge(dataframe_calendar, dataframe_customer,  how='inner', 
                  left_on=['Game_Name', 'Timeslot'], 
                  right_on = ['Game_Name', 'Timeslot'])
    
    df1 = pd.merge(dataframe_calendar, dataframe_capacity,  how='inner', 
                  left_on=['Game_Name', 'Timeslot'], 
                  right_on = ['Game_Name', 'Timeslot'])
    
    df2 = pd.merge(dataframe_calendar, dataframe_distribution,  how='inner', 
                  left_on=['Game_Name', 'Timeslot'], 
                  right_on = ['Game_Name', 'Timeslot'])

    features = ["Timeslot", "Day_of_Week", "Hour_of_Day", "Tariff", "Usage_Per_Population"]
    features1 = ["Timeslot", "Penalty"]
    features2 = ["Timeslot", "Day_of_Week", "Hour_of_Day", "Total_Consumption"]
    df = df[features]
    df1 = df1[features1]
    df2 = df2[features2]
    
    return df, df1, df2

In [ ]:
def analyse(dataframe, directory, name):
    
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    fig = plt.figure(figsize=(24, 14))
    fig.tight_layout()
    ax1 = plt.subplot(2, 2, 1)
    ax2 = plt.subplot(2, 2, 2)
    ax3 = plt.subplot(2, 2, 3)
    ax4 = plt.subplot(2, 2, 4)
    
    dataframe_week = dataframe[~dataframe.Day_of_Week.isin([6,7])]
    dataframe_weekend = dataframe[dataframe.Day_of_Week.isin([6,7])]

    dataframe_week_mean_usage = dataframe_week.groupby(['Hour_of_Day']).mean()
    dataframe_weekend_mean_usage = dataframe_weekend.groupby(['Hour_of_Day']).mean()

    dataframe_week_stdev_usage = dataframe_week.groupby(['Hour_of_Day']).std()
    dataframe_weekend_stdev_usage = dataframe_weekend.groupby(['Hour_of_Day']).std()

    ax1.plot(dataframe_week_mean_usage["Tariff"], color = "r", label='Week_Day')
    ax1.fill_between(np.arange(24),dataframe_week_mean_usage["Tariff"]-dataframe_week_stdev_usage["Tariff"], dataframe_week_mean_usage["Tariff"]+dataframe_week_stdev_usage["Tariff"] ,alpha=0.2, facecolor="r")

    ax2.plot(dataframe_weekend_mean_usage["Tariff"], color = "g", label='Week_End')
    ax2.fill_between(np.arange(24),dataframe_weekend_mean_usage["Tariff"]-dataframe_weekend_stdev_usage["Tariff"], dataframe_weekend_mean_usage["Tariff"]+dataframe_weekend_stdev_usage["Tariff"] ,alpha=0.2, facecolor="g")

    ax1.set_ylabel('Unit Tariff ($/KWh)', fontsize=18, fontweight='bold')

    ax3.plot(dataframe_week_mean_usage["Usage_Per_Population"], color = "r", label='Week_Day')
    ax3.fill_between(np.arange(24),dataframe_week_mean_usage["Usage_Per_Population"]-dataframe_week_stdev_usage["Usage_Per_Population"], dataframe_week_mean_usage["Usage_Per_Population"]+dataframe_week_stdev_usage["Usage_Per_Population"] ,alpha=0.2, facecolor="r")

    ax4.plot(dataframe_weekend_mean_usage["Usage_Per_Population"], color = "g", label='Week_End')
    ax4.fill_between(np.arange(24),dataframe_weekend_mean_usage["Usage_Per_Population"]-dataframe_weekend_stdev_usage["Usage_Per_Population"], dataframe_weekend_mean_usage["Usage_Per_Population"]+dataframe_weekend_stdev_usage["Usage_Per_Population"] ,alpha=0.2, facecolor="g")
    
    ax3.set_ylabel('Usage Per Population (KWh)', fontsize=18, fontweight='bold')
    ax3.set_xlabel('Hour', fontsize=18, fontweight='bold')
    ax4.set_xlabel('Hour', fontsize=18, fontweight='bold')
    
    ax1.tick_params(axis='both', labelsize=18)
    ax2.tick_params(axis='both', labelsize=18)
    ax3.tick_params(axis='both', labelsize=18)
    ax4.tick_params(axis='both', labelsize=18)
    
    ax1.set_ylim([0.05, 0.20])
    ax2.set_ylim([0.05, 0.20])
    
    ax1.set_xlim([1, 24])
    ax2.set_xlim([1, 24])
    ax3.set_xlim([1, 24])
    ax4.set_xlim([1, 24])
    
    ax1.set_title("Weekday Tariff Rates", fontsize=18, fontweight='bold')
    ax2.set_title("Weekend Tariff Rates", fontsize=18, fontweight='bold')
    ax3.set_title("Weekday Usage Pattern", fontsize=18, fontweight='bold')
    ax4.set_title("Weekend Usage Pattern", fontsize=18, fontweight='bold')
    
    plt.savefig(directory + name + '.png')
    plt.close()

In [ ]:
def analyse_full_game(dataframe, dataframe1, directory, name):
    
    if not os.path.exists(directory):
        os.makedirs(directory)
        
    num_rows = 3
    num_cols = 1

    fig = make_subplots(rows=num_rows, 
                        cols=num_cols,
                        subplot_titles=('Tariff Rates', 'Capacity Transactions', 'Usage Pattern'),
                        vertical_spacing=0.10)
    
    dataframe1 = dataframe1.groupby(['Timeslot'])
    dataframe11 = abs(dataframe1.sum())
    trace1 = go.Bar(name='cap_trans', x=list(dataframe1.groups.keys()), y=dataframe11['Penalty'])
    
    fig.append_trace(go.Scatter(x = dataframe["Timeslot"], y = dataframe["Tariff"], name = 'tariff'), row = 1, col = 1)
    fig.append_trace(go.Scatter(x = dataframe["Timeslot"], y = dataframe["Usage_Per_Population"], name = 'usage'), row = 3, col = 1)
    fig.append_trace(trace1, row = 2, col = 1)
    
    fig.update_yaxes(title_text="Unit Tariff ($/KWh)", row = 1)
    fig.update_yaxes(title_text="Usage Per Population (KWh)", row = 3)
    fig.update_yaxes(title_text="Penalty ($)", row = 2)
    fig.update_xaxes(title_text="Timeslot", row = 3)

    fig.update_layout(height=750, width=1500, template='ggplot2')

    for i in fig['layout']['annotations']:
        i['font'] = dict(size=14)

    fig.write_html(directory + name + '.html')

In [ ]:
def analyse_full_game_distribution_data(dataframe, dataframe1, dataframe2, directory, name):
    
    if not os.path.exists(directory):
        os.makedirs(directory)
        
    num_rows = 3
    num_cols = 1

    fig = make_subplots(rows=num_rows, 
                        cols=num_cols,
                        subplot_titles=('Tariff Rates', 'Capacity Transactions', 'Usage Pattern'),
                        vertical_spacing=0.10)
    
    dataframe1 = dataframe1.groupby(['Timeslot'])
    dataframe11 = abs(dataframe1.sum())
    trace1 = go.Bar(name='cap_trans', x=list(dataframe1.groups.keys()), y=dataframe11['Penalty'])
    
    fig.append_trace(go.Scatter(x = dataframe["Timeslot"], y = dataframe["Tariff"], name = 'tariff'), row = 1, col = 1)
    fig.append_trace(go.Scatter(x = dataframe2["Timeslot"], y = dataframe2["Total_Consumption"], name = 'usage'), row = 3, col = 1)
    fig.append_trace(trace1, row = 2, col = 1)
    
    fig.update_yaxes(title_text="Unit Tariff ($/KWh)", row = 1)
    fig.update_yaxes(title_text="Total Consuption (KWh)", row = 3)
    fig.update_yaxes(title_text="Penalty ($)", row = 2)
    fig.update_xaxes(title_text="Timeslot", row = 3)

    fig.update_layout(height=750, width=1500, template='ggplot2')

    for i in fig['layout']['annotations']:
        i['font'] = dict(size=14)

    fig.write_html(directory + name + '.html')

In [ ]:
def analyse_full_game_profit(dataframe, dataframe1, dataframe2, directory, name):
    
    if not os.path.exists(directory):
        os.makedirs(directory)
        
    dataframe1 = dataframe1.groupby(['Timeslot'])
    dataframe11 = abs(dataframe1.sum())

    df = pd.merge(dataframe, dataframe2,  how='inner', 
                  left_on=['Timeslot'], 
                  right_on = ['Timeslot'])

    df["Revenue"] = df["Total_Consumption"]*df["Tariff"]
    df = df[['Timeslot', 'Day_of_Week_x', 'Hour_of_Day_x', 'Tariff', 'Revenue']]

    start_index = df['Timeslot'][0]
    last_index = df['Timeslot'][df['Timeslot'].size - 1]

    tariff_ID = ['Tariff1', 'Tariff2', 'Tariff3', 'Tariff4', 'Tariff5', 'Tariff6', 'Tariff7', 'Tariff8', 'Tariff9']*100
    index = 0

    list2 = []

    while(start_index != (last_index+1)):

        if((start_index != 360) and (start_index-360)%168 == 0):
            index = index + 1

        list2.append(index)
        start_index += 1

    df['Tariff_ID'] = list2

    temp = df.groupby(['Tariff_ID'])
    df_ID = temp.sum()
    ID_keys = list(temp.groups.keys())
    df_ID.drop(df_ID.tail(1).index,inplace=True) # drop last n rows
    del ID_keys[-1]

    dataframe11['ID'] = ID_keys
    dataframe11['Revenue'] = list(df_ID['Revenue'])
    dataframe11['Profit'] = dataframe11['Revenue'] - dataframe11['Penalty']
    dataframe11['Label'] = tariff_ID[0:len(dataframe11)]

    dataframe111 = dataframe11.groupby(['Label']).sum()

    num_rows = 2
    num_cols = 1

    fig = make_subplots(rows=num_rows, 
                        cols=num_cols,
                        subplot_titles=('Tariff Rates', 'Profit'),
                        vertical_spacing=0.10)

    trace1 = go.Bar(name='cap_trans', x=list(dataframe1.groups.keys()), y=dataframe11['Profit'])

    fig.append_trace(go.Scatter(x = dataframe["Timeslot"], y = dataframe["Tariff"], name = 'Tariff'), row = 1, col = 1)
    fig.append_trace(trace1, row = 2, col = 1)

    fig.update_yaxes(title_text="Unit Tariff ($/KWh)", row = 1)
    fig.update_yaxes(title_text="Profit ($)", row = 2)
    fig.update_xaxes(title_text="Timeslot", row = 3)

    fig.update_layout(height=750, width=1500, template='ggplot2')

    for i in fig['layout']['annotations']:
        i['font'] = dict(size=14)

    fig.write_html(directory + name + '.html')

    dataframe111 = dataframe111[['Penalty', 'Revenue', 'Profit']]
    dataframe11 = dataframe11[['ID', 'Label', 'Penalty', 'Revenue', 'Profit']]
    dataframe11.to_csv(directory + name + '_per_week.csv')
    dataframe111.to_csv(directory + name + '_per_tariff.csv')

In [ ]:
customers = customer_info.get_targeted_customers()
# databases = ['PowerTAC2022_DR_Original_Usage', 'PowerTAC2022_DR_01Discount_Usage', 'PowerTAC2022_DR_02Discount_Usage', 'PowerTAC2022_DR_03Discount_Usage', 
#             'PowerTAC2022_DR_04Discount_Usage', 'PowerTAC2022_DR_05Discount_Usage', 'PowerTAC2022_DR_06Discount_Usage', 'PowerTAC2022_DR_07Discount_Usage', 'PowerTAC2022_DR_08Discount_Usage']
# names = ['Original_Usage', '01Discount_Usage', '02Discount_Usage', '03Discount_Usage', '04Discount_Usage', '05Discount_Usage', '06Discount_Usage', '07Discount_Usage', '08Discount_Usage']

databases = ['PowerTAC2022_DR_EX2_Discount_Usage_CapTran5', 'PowerTAC2022_DR_EX2_Discount_Usage_CapTran6', 'PowerTAC2022_DR_EX2_Discount_Usage_CapTran7',
             'PowerTAC2022_DR_EX2_Discount_Usage_CapTran8', 'PowerTAC2022_DR_EX2_Discount_Usage_CapTran9', 'PowerTAC2022_DR_EX2_Discount_Usage_CapTran10']
names = ['Test5', 'Test6', 'Test7', 'Test8', 'Test9', 'Test10']

# for customer in customers:
#     for database, name in zip(databases, names):

#         print(customer, " ", database, " ", name)
        
#         try:
#             df, df1, df2 = get_dataframe(database, customer)
#             directory = storage_path + customer + '/'
#             analyse_full_game(df, df1, directory, name)
#         except Exception as e:
#             print(e)
            
# for database, name in zip(databases, names):
        
#         try:
#             df, df1, df2 = get_dataframe(database, 'BrooksideHomes')
#             directory = storage_path + 'DistributionData/'
#             analyse_full_game_distribution_data(df, df1, df2, directory, name)
#         except Exception as e:
#             print(e)

for database, name in zip(databases, names):
        
        try:
            df, df1, df2 = get_dataframe(database, 'BrooksideHomes')
            directory = storage_path + 'DistributionData/'
            analyse_full_game_profit(df, df1, df2, directory, name)
        except Exception as e:
            print(e)